In [2]:
import os
import glob
import pickle
from tqdm import tqdm # Per una bella barra di avanzamento
import numpy as np

from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

# 1. Caricare il modello InceptionV3 pre-addestrato su ImageNet
print("Caricamento del modello InceptionV3...")
base_model = InceptionV3(weights='imagenet')

Caricamento del modello InceptionV3...


In [ ]:
# 2. Creare un nuovo modello che prende l'input di InceptionV3 e
#    restituisce l'output del layer 'avg_pool'.
#    Questo layer produce un vettore di 2048 feature per ogni immagine.
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
print("Modello per l'estrazione delle feature creato.")

Modello per l'estrazione delle feature creato.


In [4]:
# 3. Definire una funzione per processare una singola immagine
def extract_feature(img_path, model):
    """
    Carica un'immagine, la preprocessa per InceptionV3 e ne estrae le feature.
    """
    try:
        # InceptionV3 richiede immagini di dimensione 299x299
        img = image.load_img(img_path, target_size=(299, 299))
        
        # Converte l'immagine in un array NumPy
        x = image.img_to_array(img)
        
        # Aggiunge una dimensione extra per il "batch" (richiesto dal modello)
        x = np.expand_dims(x, axis=0)
        
        # Preprocessa l'input nello stesso modo in cui InceptionV3 è stato addestrato
        x = preprocess_input(x)
        
        # Esegue la predizione (estrazione feature)
        feature = model.predict(x, verbose=0)
        
        # L'output di avg_pool è (1, 1, 2048). Lo rimodelliamo in un vettore semplice (2048,).
        return feature.flatten() # .flatten() o .reshape(2048,) sono equivalenti qui

    except Exception as e:
        print(f"\nErrore durante l'elaborazione di {img_path}: {e}")
        return None


In [5]:
# 4. Specificare il percorso delle immagini
images_path = r"..\data\Flickr8k_Dataset\Images" # Usa il percorso corretto
if not os.path.exists(images_path):
    raise FileNotFoundError(f"La cartella delle immagini non è stata trovata in: {images_path}")

In [6]:
# Trova tutti i file .jpg nella cartella
image_files = glob.glob(os.path.join(images_path, "*.jpg"))
print(f"Trovate {len(image_files)} immagini da processare.")

# 5. Creare la cartella di output se non esiste
output_folder = "features"
os.makedirs(output_folder, exist_ok=True)

Trovate 8091 immagini da processare.


In [7]:
# 6. Eseguire l'estrazione per tutte le immagini e salvare in un dizionario
#    Il dizionario mapperà image_id -> vettore_feature
features_dict = {}
for img_path in tqdm(image_files, desc="Estrazione feature"):
    image_id = os.path.basename(img_path).split('.')[0]
    feature_vector = extract_feature(img_path, model)
    
    # Aggiungi al dizionario solo se l'estrazione è andata a buon fine
    if feature_vector is not None:
        features_dict[image_id] = feature_vector

Estrazione feature: 100%|██████████| 8091/8091 [46:56<00:00,  2.87it/s]  


In [8]:
# 7. Salvare il dizionario in un file .pkl per un uso futuro
output_path = os.path.join(output_folder, "image_features.pkl")
with open(output_path, "wb") as f:
    pickle.dump(features_dict, f)

print(f"\n✔️ Estrazione completata. {len(features_dict)} feature salvate in '{output_path}'.")


✔️ Estrazione completata. 8091 feature salvate in 'features\image_features.pkl'.
